In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [19]:
test_chrome = "C:/Users/Dell/Downloads/chrome-win64/chrome-win64/chrome.exe"
options = Options()
options.binary_location = test_chrome
driver_path = "C:/Users/Dell/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe"
service = Service(driver_path)

driver= webdriver.Chrome(service=service, options=options)
action = ActionChains(driver)
driver.get('https://amazon.in')
search = driver.find_element(By.ID, 'twotabsearchtextbox')

'''time.sleep(2)
action.click(search).key_down(Keys.SHIFT).send_keys('love python').key_up(Keys.SHIFT)
time.sleep(2)
action.send_keys(Keys.ENTER)
time.sleep(2)
search.send_keys('Laptop')
time.sleep(2)
search.send_keys(Keys.ENTER)'''

action.click(search).send_keys('Laptop').send_keys(Keys.ENTER).perform()
time.sleep(3)
wait = WebDriverWait(driver, 10)
#products = driver.find_elements(By.CLASS_NAME, 'puisg-row')
#print(products)

page_count = 0
max_pages = 5
laptop_data = []

# Loop through pages
while True:
    # Wait for search results to load
    wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(@class, 's-main-slot')]//div[@data-component-type='s-search-result']")))

    # Collect the titles, prices, and reviews on the current page
    laptops = driver.find_elements(By.XPATH, "//div[contains(@class, 's-main-slot')]//div[@data-component-type='s-search-result']")
    
    for laptop in laptops:
        try:
            # Title
            title = laptop.find_element(By.XPATH, ".//h2/a/span").text

            # Price (Discounted)
            try:
                discounted_price = laptop.find_element(By.XPATH, ".//span[@class='a-price-whole']").text
            except:
                discounted_price = "Price not available"

            # Average Review Score
            try:
                average_review = laptop.find_element(By.XPATH, ".//span[@class='a-icon-alt']").get_attribute("innerHTML").split()[0]
            except:
                average_review = "No reviews"

            # Append data to list
            laptop_data.append({
                'Title': title,
                'Discounted Price': discounted_price,
                'Average Review': f"{average_review} out of 5" if average_review != "No reviews" else "No reviews"
            })
        except Exception as e:
            print(f"Error while extracting data for a laptop: {e}")
            continue
    #page_count +=1
    # Move to the next page if available
    try:
        next_button = driver.find_element(By.XPATH, "//a[contains(@class, 's-pagination-next')]")
        if 'disabled' in next_button.get_attribute("class"):
            break  # No more pages, exit the loop
        else:
            next_button.click()  # Go to the next page
            time.sleep(3)  # Wait for page to load
    except:
        break  # Exit if next button not found

# Close the driver
driver.quit()


In [20]:
import pandas as pd
df= pd.DataFrame(laptop_data)

In [23]:
df.head()


,Title,Discounted Price,Average Review
0,"Dell [Smartchoice] Core i3-1215U, 12th Gen (8G...","38,490",3.7 out of 5
1,Acer Aspire Lite 13th Gen Intel Core i3-1305U ...,"33,990",3.9 out of 5
2,"Apple MacBook Air Laptop: Apple M1 chip, 13.3-...","73,990",4.6 out of 5
3,"HP Laptop 15s, 12th Gen Intel Core i3, 15.6-in...","37,990",4.0 out of 5
4,Acer Chromebook CB315-4H Intel Celeron N4500 (...,"15,990",3.5 out of 5


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Title             386 non-null    object
 1   Discounted Price  386 non-null    object
 2   Average Review    386 non-null    object
dtypes: object(3)
memory usage: 9.2+ KB
